In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
# change the font size of plot
plt.rcParams.update({'font.size': 16})

In [12]:
import struct
print(struct.calcsize("P") * 8)

64


In [3]:
train = pd.read_csv('data/train.csv')
# test = pd.read_csv('data/test.csv')

In [4]:
#del test
#del weather_test_prepared_df

In [5]:
# read data from file:
building_prepared_df = pd.read_csv('C:\\data\\Kaggle_ASHRAE_energy\\building_prepared_df.csv')
weather_train_prepared_df = pd.read_csv('C:\\data\\Kaggle_ASHRAE_energy\\weather_train_prepared_df.csv')
# weather_test_prepared_df = pd.read_csv('C:\\data\\Kaggle_ASHRAE_energy\\weather_test_prepared_df.csv')

In [6]:
building_prepared_df.shape, weather_train_prepared_df.shape

((1449, 1467), (139773, 11))

In [7]:
train.shape

(20216100, 4)

In [8]:
building_prepared_df.head()

,site_id,building_id,0,1,2,3,4,5,6,7,...,1455,1456,1457,1458,1459,1460,1461,1462,1463,1464
0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008173
1,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002786
2,0,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005822
3,0,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.026754
4,0,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.132985


In [9]:
weather_train_prepared_df.columns = ['site_id','timestamp','airtemp','sealev','dewtemp','windsp',
                                     'North','West','South','nowind','dewair']

In [10]:
weather_train_prepared_df.head()

,site_id,timestamp,airtemp,sealev,dewtemp,windsp,North,West,South,nowind,dewair
0,0,2016-01-01 00:00:00,0.698187,0.628816,0.891410,0.000000,0.0,0.0,0.0,1.0,0.960000
1,0,2016-01-01 01:00:00,0.690415,0.634921,0.909238,0.061983,1.0,0.0,0.0,0.0,0.973473
2,0,2016-01-01 02:00:00,0.669689,0.634921,0.909238,0.000000,0.0,0.0,0.0,1.0,0.986156
3,0,2016-01-01 03:00:00,0.647668,0.633700,0.901135,0.000000,0.0,0.0,0.0,1.0,0.995871
4,0,2016-01-01 04:00:00,0.633420,0.632479,0.891410,0.107438,0.0,1.0,0.0,0.0,1.000000


### Start cleaning the Y (meter reading) data.

get the site_id and the data order from building and weather

In [15]:
# merge site_id, building_id, timestamp data from building and weather :
site_building_ts = pd.merge(left=building_prepared_df[['site_id','building_id']], 
                               right = weather_train_prepared_df[['site_id','timestamp']], 
                               left_on='site_id', right_on = 'site_id')
site_building_ts

,site_id,building_id,timestamp
0,0,0,2016-01-01 00:00:00
1,0,0,2016-01-01 01:00:00
2,0,0,2016-01-01 02:00:00
3,0,0,2016-01-01 03:00:00
4,0,0,2016-01-01 04:00:00
...,...,...,...
12676161,15,1448,2016-12-31 19:00:00
12676162,15,1448,2016-12-31 20:00:00
12676163,15,1448,2016-12-31 21:00:00
12676164,15,1448,2016-12-31 22:00:00


#### merge table
merge dataset above with meter_reading data and split into 4 dataset by different meters

In [22]:
train_set0 = pd.merge(left= site_building_ts, right = train.query('meter == 0').drop('meter',axis=1), 
                      left_on=['building_id','timestamp'], right_on = ['building_id','timestamp'])
train_set1 = pd.merge(left= site_building_ts, right = train.query('meter == 1').drop('meter',axis=1), 
                      left_on=['building_id','timestamp'], right_on = ['building_id','timestamp'])
train_set2 = pd.merge(left= site_building_ts, right = train.query('meter == 2').drop('meter',axis=1), 
                      left_on=['building_id','timestamp'], right_on = ['building_id','timestamp'])
train_set3 = pd.merge(left= site_building_ts, right = train.query('meter == 3').drop('meter',axis=1), 
                      left_on=['building_id','timestamp'], right_on = ['building_id','timestamp'])

In [23]:
# I can get the same data order from below query too, but missing site_id
# train.sort_values(by = ['building_id','meter','timestamp'])

query shows lots of zeros for meter0 reading

In [26]:
train_set0.head(30)

,site_id,building_id,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,0,0,2016-01-01 01:00:00,0.0
2,0,0,2016-01-01 02:00:00,0.0
3,0,0,2016-01-01 03:00:00,0.0
4,0,0,2016-01-01 04:00:00,0.0
5,0,0,2016-01-01 05:00:00,0.0
6,0,0,2016-01-01 06:00:00,0.0
7,0,0,2016-01-01 07:00:00,0.0
8,0,0,2016-01-01 08:00:00,0.0
9,0,0,2016-01-01 09:00:00,0.0


below query also tells me there is a scenario that some building didn't have the meter initially, but install it sometime after. I should run that 0 meter deletion on ALL buildings and all meters.not only on some

In [48]:
train_set2.query('building_id == 745')

,site_id,building_id,timestamp,meter_reading
0,6,745,2016-01-01 01:00:00,0.0000
1,6,745,2016-01-01 02:00:00,0.0000
2,6,745,2016-01-01 03:00:00,0.0000
3,6,745,2016-01-01 04:00:00,0.0000
4,6,745,2016-01-01 05:00:00,0.0000
...,...,...,...,...
3372,6,745,2016-12-31 13:00:00,2.9249
3373,6,745,2016-12-31 19:00:00,9.8624
3374,6,745,2016-12-31 21:00:00,27.0749
3375,6,745,2016-12-31 22:00:00,1.4999


In [51]:
# create empty DataFrame, 
# I am going to put the number of zeros and indexes with non zero for each building into this DF
building_non_zero = pd.DataFrame(columns = ['building_id','zeros','start_from','non_zero_index'])
building_non_zero

,building_id,zeros,start_from,non_zero_index


In [54]:
for each in train_set0.building_id.unique():
    query_temp = 'building_id == ' + str(each)
    build_data = train_set0.query(query_temp)
    build_start = build_data.head(1).index.values[0]
    index_first0 = build_start
    cumsum_non0_rowcount = np.cumsum(build_data.meter_reading != 0)
    counts = 1

    # not only just check first non-0, but also check the next two rows, 
    # if they are 0, keep looking for the next non-0 
    while True:
        index_first0= build_data.loc[(cumsum_non0_rowcount == counts)].head(1).index.values[0]
        if train_set0.meter_reading.iloc[(index_first0+1)] > 0 and train_set0.meter_reading.iloc[(index_first0+2)] > 0:
            break
        counts+=1
    zeros_temp = index_first0 - build_start
    building_non_zero = building_non_zero.append({'building_id':each,'zeros':zeros_temp,'start_from':build_start,
                                                  'non_zero_index':index_first0}, ignore_index=True)

In [55]:
building_non_zero

,building_id,zeros,start_from,non_zero_index
0,0,3378,0,3378
1,1,3378,8784,12162
2,2,3378,17568,20946
3,3,3378,26352,29730
4,4,3378,35136,38514
...,...,...,...,...
1408,1444,0,11981598,11981598
1409,1445,0,11988748,11988748
1410,1446,22,11995901,11995923
1411,1447,0,12003077,12003077


In [69]:
del building_non_zero

below shows the buildings belong to site_id=0

In [66]:
building_prepared_df.query('site_id==0').building_id.unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104], dtype=int64)

below two queries show after building_id > 104, there are very few building starting with 0 meter_reading for meter0. ---- This means something wrong on the meter 0 meter_reading data for most of buildings in site_id = 0 (also see the query after)

In [67]:
np.cumsum(building_non_zero.iloc[105:].zeros != 0)

105      0
106      0
107      0
108      0
109      0
        ..
1408    21
1409    21
1410    22
1411    22
1412    22
Name: zeros, Length: 1308, dtype: int32

In [68]:
np.cumsum(building_non_zero.iloc[:105].zeros != 0)

0        1
1        2
2        3
3        4
4        5
      ... 
100    100
101    101
102    102
103    103
104    104
Name: zeros, Length: 105, dtype: int32

I should loop all the meter reading data, find out exceptional zeros and execute deletion

In [49]:
meterlist = [train_set0,train_set1,train_set2,train_set3]

In [70]:
for meter in meterlist:
    # find out zeros and the indexes:
    building_non_zero = pd.DataFrame(columns = ['building_id','zeros','start_from','non_zero_index'])
    for each in meter.building_id.unique():
        query_temp = 'building_id == ' + str(each)
        build_data = meter.query(query_temp)
        build_start = build_data.head(1).index.values[0]
        index_first0 = build_start
        cumsum_non0_rowcount = np.cumsum(build_data.meter_reading != 0)
        counts = 1

        # not only just check first non-0, but also check the next two rows, 
        # if they are 0, keep looking for the next non-0 
        while True:
            index_first0= build_data.loc[(cumsum_non0_rowcount == counts)].head(1).index.values[0]
            if meter.meter_reading.iloc[(index_first0+1)] > 0 and meter.meter_reading.iloc[(index_first0+2)] > 0:
                break
            counts+=1
        zeros_temp = index_first0 - build_start
        building_non_zero = building_non_zero.append({'building_id':each,'zeros':zeros_temp,'start_from':build_start,
                                                      'non_zero_index':index_first0}, ignore_index=True)
    # execute deletion:
    for each in building_non_zero.building_id:
        query_temp = 'building_id =='+ str(each)
        building_row = building_non_zero.query(query_temp)
        meter = meter.drop(list(range(building_row.start_from.values[0],building_row.non_zero_index.values[0])))


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
train_set0